

# 🎬 Advanced Movie Recommendation System: Colab Edition

This notebook implements a **Hybrid Recommendation Engine** combining:

1. **Collaborative Filtering:** Uses SVD (Matrix Factorization) to learn user preferences.
2. **Content-Based Filtering:** Matches genres and tags.
3. **Semantic Search:** Uses `SentenceTransformers` and `FAISS` to understand natural language queries (e.g., "Sad movies about space").

---

### **Step 1: Install Dependencies**

We need libraries for matrix factorization (`scikit-surprise`), vector search (`faiss-cpu`), and embeddings (`sentence-transformers`).

In [1]:
# @title 1. Install Libraries
!pip install scikit-surprise sentence-transformers faiss-cpu pandas "numpy<2" scikit-learn
print("✅ Libraries Installed Successfully")

✅ Libraries Installed Successfully


---

### **Step 2: Load & Prepare Data**

We will use the **MovieLens Small Dataset** for this demonstration because it loads instantly. We will also synthesize some "plot summaries" since the raw MovieLens data only has titles and genres.

In [2]:
# @title 2. Load and Preprocess Data
import pandas as pd
import numpy as np
import requests
import zipfile
import io

# Download MovieLens Small Dataset
print("Downloading dataset...")
url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

# Load CSVs
movies = pd.read_csv("./ml-latest-small/movies.csv")
ratings = pd.read_csv("./ml-latest-small/ratings.csv")

# Create synthetic "Overview" text because MovieLens only has titles/genres
# In a real app, you would merge this with TMDB/IMDB data
movies['overview'] = movies['title'] + " " + movies['genres'].str.replace('|', ' ') + " Movie"

print(f"✅ Data Loaded: {movies.shape[0]} movies, {ratings.shape[0]} ratings.")
print("Sample Movie:")
display(movies.head(2))

✅ Data Loaded: 9742 movies, 100836 ratings.
Sample Movie:


,movieId,title,genres,overview
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995) Adventure Animation Children ...
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji (1995) Adventure Children Fantasy Movie


---

### **Step 3: Collaborative Filtering (User Preferences)**

We use the **SVD (Singular Value Decomposition)** algorithm from the `Surprise` library. This attempts to predict how a user would rate a movie they haven't seen yet based on their rating history.

In [3]:
# @title 3. Train Collaborative Filtering Model (SVD)
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split

# 1. Prepare data for Surprise
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# 2. Train the model (using full dataset for this demo)
trainset = data.build_full_trainset()
svd_model = SVD()
svd_model.fit(trainset)

print("✅ Collaborative Filtering Model Trained!")

# Test a prediction
user_id = 1
movie_id = 50 # The Usual Suspects
pred = svd_model.predict(user_id, movie_id)
print(f"Predicted score for User {user_id} on Movie {movie_id}: {pred.est:.2f}/5.0")

✅ Collaborative Filtering Model Trained!
Predicted score for User 1 on Movie 50: 5.00/5.0


---

### **Step 4: Semantic Search Engine (FAISS + Embeddings)**

This is the "Advanced" part. We convert movie descriptions into mathematical vectors. This allows users to search by *feeling* or *vibe* rather than just keywords.

In [4]:
# @title 4. Build Vector Search Index (FAISS)
from sentence_transformers import SentenceTransformer
import faiss

# 1. Load a lightweight BERT model
# 'all-MiniLM-L6-v2' is fast and good for semantic similarity
bert_model = SentenceTransformer('all-MiniLM-L6-v2')

# 2. Create Embeddings for all movies
print("⏳ Generating embeddings (this may take 30-60 seconds)...")
movie_overviews = movies['overview'].tolist()
embeddings = bert_model.encode(movie_overviews)

# 3. Build FAISS Index for fast searching
dimension = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(np.array(embeddings))

print(f"✅ FAISS Index created with {faiss_index.ntotal} vectors.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

⏳ Generating embeddings (this may take 30-60 seconds)...
✅ FAISS Index created with 9742 vectors.


---

### **Step 5: The Hybrid Recommendation Logic**

This function combines the scores.

* **Vector Score:** How well does the movie match the user's text query?
* **Collaborative Score:** How much do we think this specific user will like the movie?

In [5]:
# @title 5. Define Hybrid Recommendation Function
def hybrid_recommendation(user_id, query_text, top_k=10):
    # --- A. Semantic Search (Content/Query Match) ---
    query_vector = bert_model.encode([query_text])

    # Search FAISS for nearest neighbors (lower distance = better match)
    distances, indices = faiss_index.search(query_vector, top_k * 5) # Get more candidates to filter

    candidate_indices = indices[0]
    candidate_distances = distances[0]

    recommendations = []

    # --- B. Hybrid Scoring Loop ---
    for i, idx in enumerate(candidate_indices):
        movie_id = movies.iloc[idx]['movieId']
        title = movies.iloc[idx]['title']
        genres = movies.iloc[idx]['genres']

        # 1. Get Semantic Score (normalize distance to 0-1 scale approx)
        # In L2 distance, 0 is perfect match. We invert it for scoring.
        semantic_score = 1 / (1 + candidate_distances[i])

        # 2. Get Collaborative Score (Predicted Rating 1-5)
        # If user is generic (ID 0), assume neutral rating 3.0
        if user_id > 0:
            cf_prediction = svd_model.predict(user_id, movie_id).est
        else:
            cf_prediction = 3.0

        # Normalize CF score to 0-1 (5.0 -> 1.0)
        cf_score_norm = cf_prediction / 5.0

        # --- C. Weighted Final Score ---
        # Weights: 60% Semantic Match, 40% User Preference
        final_score = (0.6 * semantic_score) + (0.4 * cf_score_norm)

        recommendations.append({
            "Title": title,
            "Genres": genres,
            "Semantic Score": round(semantic_score, 3),
            "Predicted User Rating": round(cf_prediction, 2),
            "Final Hybrid Score": round(final_score, 3)
        })

    # Sort by Final Hybrid Score
    rec_df = pd.DataFrame(recommendations)
    rec_df = rec_df.sort_values(by="Final Hybrid Score", ascending=False).head(top_k)

    return rec_df

print("✅ Hybrid Engine Ready.")

✅ Hybrid Engine Ready.


---

### **Step 6: Interactive Demo**

Use this widget to test your engine immediately.

In [6]:
# @title 6. 🚀 Test the Recommender
import ipywidgets as widgets
from IPython.display import display

print("💡 Try queries like: 'Toy Story', 'Space adventure', 'Scary movie with ghosts'")

# UI Elements
user_id_input = widgets.IntText(value=1, description='User ID:')
query_input = widgets.Text(value='Toy Story', description='Query:')
button = widgets.Button(description="Recommend")
output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        print(f"🔎 Searching for: '{query_input.value}' for User {user_id_input.value}...")
        results = hybrid_recommendation(user_id_input.value, query_input.value)
        display(results)

button.on_click(on_button_click)
display(user_id_input, query_input, button, output)

💡 Try queries like: 'Toy Story', 'Space adventure', 'Scary movie with ghosts'


IntText(value=1, description='User ID:')

Text(value='Toy Story', description='Query:')

Button(description='Recommend', style=ButtonStyle())

Output()

In [8]:

import pickle
import faiss
from google.colab import files

print("💾 Saving models...")

# 1. Save the SVD Model
with open('svd_model.pkl', 'wb') as f:
    pickle.dump(svd_model, f)

# 2. Save the Movie Metadata (The dataframe with titles/genres)
with open('movies_metadata.pkl', 'wb') as f:
    pickle.dump(movies, f)

# 3. Save the FAISS Index (The embeddings)
faiss.write_index(faiss_index, "faiss_index.bin")

print("✅ Files saved. Zipping for download...")

# Zip them together
!zip models_export.zip svd_model.pkl movies_metadata.pkl faiss_index.bin

# Trigger the browser download
files.download('models_export.zip')

💾 Saving models...
✅ Files saved. Zipping for download...
  adding: svd_model.pkl (deflated 24%)
  adding: movies_metadata.pkl (deflated 68%)
  adding: faiss_index.bin (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>